In [ ]:
import socket, threading
import time

class HiloVigilante(threading.Thread):
    #funcion para la instancia de hilos con los parametros
    bandera = False
    SERVER="" #ip de localhost
    PORT=0 #Puerto a utilizar
    cliente=None
    def __init__(self, cliente_direction, cliente_socket):
        #creacion de hilo nuevo
        self.SERVER = cliente_direction
        self.PORT = cliente_socket
        self.cliente=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        self.cliente.connect((self.SERVER, self.PORT)) #Conexion al servidor utilizando ip y puerto
        threading.Thread.__init__(self)
        self.csocket = cliente_socket
        #Mensaje de conexion cliente1 y hora
    def run(self):
        datos_entrada=self.cliente.recv(1024) #recepcion de datos del servidor en 1024 caracteres
        bienvenida=datos_entrada.decode("UTF-8")
        while True:
            self.cliente.sendall(bytes('Llega','UTF-8')) #Envio de mensaje al servidor esperando respuesta
            datos_entrada=self.cliente.recv(1024) #Espera por una respuesta del servidor
            mensaje=datos_entrada.decode()
            if mensaje=='Llego':
                self.bandera=True    
            time.sleep(1)
            
    def close(self):
        cliente.close()
        

In [ ]:
#Librerias a utilizar
import socket, threading
import time

class HiloGenerador(threading.Thread):

    SERVERS=["127.0.0.2","127.0.0.3"] #ip de los servidores activos
    PORT=8080 #Puerto a utilizar
    conexiones=[]
    numServers=2
    principal=["0.0.0.0",0]
    
    def __init__(self):
        #creacion de hilo nuevo
        threading.Thread.__init__(self)
    def run(self):
        #Crea hilos vigilantes que sirven para verificar el estado de los servidores activos
        for i in range (self.numServers):
            self.conexiones.append(HiloVigilante(self.SERVERS[i],self.PORT))
            self.conexiones[i].start()
            
        while True:
            for i in self.conexiones:
                print("Server: ",i.SERVER," ¿Activo?: ",i.bandera)
                if(i.bandera==True):
                    self.principal=[i.SERVER,i.PORT]
                i.bandera=False
            time.sleep(5)

In [ ]:
#Inicializacion de variables
LOCALHOST="127.0.0.1" #ip de localhost, se usa una dirección diferente a los servidores para evitar conflictos
PORT=8081 #puerto a utilizar se utiliza un puerto diferente para evitar crean conflicto con los servidores

server = socket.socket(socket.AF_INET,socket.SOCK_STREAM) #tipo de servidor a utilizar
server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) #Activacion de uso de servidor
server.bind((LOCALHOST, PORT)) #Creacion de servidor
#Mensajes de inicio de servidor
print("Servidor iniciado")
print("esperando peticiones de los clientes...")
#Crea el hilo generador encargado de generar los hilos de enalce a los servidores
conector=HiloGenerador()
conector.start()
while True:
    #Numero de clientes a escuchar
    server.listen(2)
    #Ontencion de socket y direccion del cliente
    cliente_sock, cliente_direccion = server.accept()
    #Obtiene la dirección y el puerto del servidor activo más reciente y lo envía al usuario luego corta la comunicación
    host=conector.principal
    print("Cliente ip: ",cliente_direccion," redireccionando a: ",host[0])
    #Enviando la direccion IP a donde el usuario debe enlazarse
    cliente_sock.send((host[0]+" "+str(host[1])).encode("UTF-8"))
    cliente_sock.close()